In [ ]:
import sys
sys.path += ["../"]

In [ ]:
import os
import glob
import numpy as np
import uproot
import time
import matplotlib

fig_width = 5
fig_height = 4
params = {
          #'backend': 'notebook',
          'text.latex.preamble': [r'\usepackage{gensymb}'],
          'axes.labelsize': 8, # fontsize for x and y labels (was 10)
          'axes.titlesize': 8,
          'font.size': 8, # was 10
          'text.usetex': False,
          'figure.figsize': [fig_width,fig_height],
          'font.family': 'serif'
}

import matplotlib.pyplot as plt
from collections import OrderedDict

In [ ]:
import hepaccelerate

In [ ]:
import pickle

In [ ]:
def midpoints(arr):
    return arr[:-1] + np.diff(arr)/2.0

def plot_hist_step(ax, edges, contents, errors, kwargs_step={}, kwargs_errorbar={}):
    line = histstep(ax, edges, contents, **kwargs_step)
    #ax.errorbar(midpoints(edges), contents, errors, lw=0, elinewidth=1, color=line.get_color()[0], **kwargs_errorbar)
    
def histstep(ax, edges, contents, **kwargs):
    ymins = []
    ymaxs = []
    xmins = []
    xmaxs = []
    for istep in range(len(edges)-1):
        xmins += [edges[istep]]
        xmaxs += [edges[istep+1]]
        ymins += [contents[istep]]
        if istep + 1 < len(contents):
            ymaxs += [contents[istep+1]]

    if not "color" in kwargs:
        kwargs["color"] = next(ax._get_lines.prop_cycler)['color']

    ymaxs += [ymaxs[-1]]
    l0 = ax.hlines(ymins, xmins, xmaxs, **kwargs)
    l1 = ax.vlines(xmaxs, ymins, ymaxs, color=l0.get_color(), linestyles=l0.get_linestyle())
    return l0


In [ ]:
def plot_hist_ratio(hists_mc, hist_data,
        total_err_stat=None,
        total_err_stat_syst=None,
        figure=None, **kwargs):

    xbins = kwargs.get("xbins", None)

    if not figure:
        figure = plt.figure(figsize=(4,4), dpi=100)

    ax1 = plt.axes([0.0, 0.23, 1.0, 0.8])

    hmc_tot = np.zeros_like(hist_data.contents)
    hmc_tot2 = np.zeros_like(hist_data.contents)

    edges = hist_data.edges
    if xbins == "uniform":
        edges = np.arange(len(hist_data.edges))


    ax1.errorbar(
        midpoints(edges), hist_data.contents,
        np.sqrt(hist_data.contents_w2), marker="o", lw=0,
        elinewidth=1.0, color="black", ms=3, label="data")
    
    for h in hists_mc:
#         plot_hist_step(ax1, edges, hmc_tot + h.contents,
#             np.sqrt(hmc_tot2 + h.contents_w2),
#             kwargs_step={"label": getattr(h, "label", None), "color": getattr(h, "color", None)}
#         )

        b = ax1.bar(midpoints(edges), h.contents, np.diff(edges), hmc_tot,
            edgecolor=getattr(h, "color", None),
            facecolor=getattr(h, "color", None),
            label=getattr(h, "label", None))
        hmc_tot += h.contents
        hmc_tot2 += h.contents_w2

    ax1.errorbar(
        midpoints(edges), hist_data.contents,
        np.sqrt(hist_data.contents_w2), marker="o", lw=0,
        elinewidth=1.0, color="black", ms=3)

    if not (total_err_stat_syst is None):
        #plt.bar(midpoints(edges), 2*total_err_stat_syst, bottom=hmc_tot - total_err_stat_syst,
        #        width=np.diff(edges), hatch="////", color="gray", fill=None, lw=0)
        histstep(ax1, edges, hmc_tot + total_err_stat_syst, color="blue", linewidth=0.5, linestyle="--", label="stat+syst")
        histstep(ax1, edges, hmc_tot - total_err_stat_syst, color="blue", linewidth=0.5, linestyle="--")

    if not (total_err_stat is None):
        #plt.bar(midpoints(edges), 2*total_err_stat, bottom=hmc_tot - total_err_stat,
        #        width=np.diff(edges), hatch="\\\\", color="gray", fill=None, lw=0)
        histstep(ax1, edges, hmc_tot + total_err_stat, color="gray", linewidth=0.5, linestyle="--", label="stat")
        histstep(ax1, edges, hmc_tot - total_err_stat, color="gray", linewidth=0.5, linestyle="--")

    if kwargs.get("do_log", False):
        ax1.set_yscale("log")
        ax1.set_ylim(1, 100*np.max(hist_data.contents))
    else:
        ax1.set_ylim(0, 2*np.max(hist_data.contents))

    #ax1.get_yticklabels()[-1].remove()

    ax2 = plt.axes([0.0, 0.0, 1.0, 0.16], sharex=ax1)

    ratio = hist_data.contents / hmc_tot
    ratio_err = np.sqrt(hist_data.contents_w2) /hmc_tot
    ratio[np.isnan(ratio)] = 0

    plt.errorbar(
        midpoints(edges), ratio, ratio_err,
        marker="o", lw=0, elinewidth=1, ms=3, color="black")

    if not (total_err_stat_syst is None):
        ratio_up = (hmc_tot + total_err_stat_syst) / hmc_tot
        ratio_down = (hmc_tot - total_err_stat_syst) / hmc_tot
        ratio_down[np.isnan(ratio_down)] = 1
        ratio_down[np.isnan(ratio_up)] = 1
        histstep(ax2, edges, ratio_up, color="blue", linewidth=0.5, linestyle="--")
        histstep(ax2, edges, ratio_down, color="blue", linewidth=0.5, linestyle="--")

    if not (total_err_stat is None):
        ratio_up = (hmc_tot + total_err_stat) / hmc_tot
        ratio_down = (hmc_tot - total_err_stat) / hmc_tot
        ratio_down[np.isnan(ratio_down)] = 1
        ratio_down[np.isnan(ratio_up)] = 1
        histstep(ax2, edges, ratio_up, color="gray", linewidth=0.5, linestyle="--")
        histstep(ax2, edges, ratio_down, color="gray", linewidth=0.5, linestyle="--")


    plt.ylim(0.5, 1.5)
    plt.axhline(1.0, color="black")

    if xbins == "uniform":
        print(hist_data.edges)
        ax1.set_xticks(edges)
        ax1.set_xticklabels(["{0:.2f}".format(x) for x in hist_data.edges])

    ax1.set_xlim(min(edges), max(edges))
    xlim = kwargs.get("xlim", None)
    if not xlim is None:
        ax1.set_xlim(*xlim)

    ylim = kwargs.get("ylim", None)
    if not ylim is None:
        ax1.set_ylim(*ylim)

    return ax1, ax2

In [ ]:
with open("../data/out.pkl", "rb") as fi:
    results = pickle.load(fi)
hists = results["hists"]

In [ ]:
hists["DYJetsToLL"]["hist__nmu1_njetge3_nbjetge2__sumpt"]

In [ ]:
histname = "hist__nmu1_njetge3_nbjetge2__sumpt"
skey = "nominal"
mc_samps = [
    'DYJetsToLL',
    'W1JetsToLNu', 'W2JetsToLNu', 'W3JetsToLNu',
    'TTJets_FullLeptMGDecays', 'TTJets_Hadronic', 'TTJets_SemiLeptMGDecays',
]

colors = {
    "DYJetsToLL": "green",
    "TTJets_FullLeptMGDecays": "blue",
    "TTJets_Hadronic": "darkblue",
    "TTJets_SemiLeptMGDecays": "lightblue",
    "W1JetsToLNu": "red",
    "W2JetsToLNu": "orange",
    "W3JetsToLNu": "pink",
}

#this is the total amount of data in inverse pb
#we use an approximate placeholder value here
lumi = 10000.0

#ad-hoc correction factor on MC to account for various unaccounted detailed MC corrections
mc_sf = 0.8

#in picobarns
#from https://escholarship.org/content/qt0gt3c4cr/qt0gt3c4cr.pdf?t=o4y9h9&nosplash=7aa805932d08bd7f9078f6a6e6034fc6
cross_sections = {
    "TTJets_SemiLeptMGDecays": 102.50,
    "TTJets_Hadronic": 106.93,
    "TTJets_FullLeptMGDecays": 24.56,
    "DYJetsToLL": 3532.8,
    "W1JetsToLNu": 6663,
    "W2JetsToLNu": 2159,
    "W3JetsToLNu": 640,
}

shape_systematics = ["jec0"]

hmc = {s: hists[s][histname][skey] for s in mc_samps}
hd = hists["SingleMu"][histname][skey]

weight_xs = {}
for s in mc_samps:
    weight_xs[s] = mc_sf * lumi * cross_sections[s] * (1.0/results["numevents"][s])
    hmc[s] *= weight_xs[s]
    hmc[s].label = s
    hmc[s].color = colors[s]

hist_template = 0*hd
htot_nominal = sum(hmc.values(), hist_template)
htot_variated = {}
hdelta_quadrature = np.zeros_like(hist_template.contents)

for sdir in ["__up", "__down"]:
    for unc in shape_systematics:
        htot_variated[unc + sdir] = sum([
            hists[mc_samp][histname][unc + sdir]* weight_xs[mc_samp] for mc_samp in mc_samps
        ], hist_template)
        hdelta_quadrature += (htot_nominal.contents - htot_variated[unc+sdir].contents)**2

hdelta_quadrature_stat = np.sqrt(htot_nominal.contents_w2)
hdelta_quadrature_stat_syst = np.sqrt(hdelta_quadrature_stat**2 + hdelta_quadrature)

ax1, ax2 = plot_hist_ratio(
    [hmc[s] for s in mc_samps],
    hd,
    total_err_stat=hdelta_quadrature_stat,
    total_err_stat_syst=hdelta_quadrature_stat_syst
)
ax1.legend(fontsize=6, frameon=False)
ax1.set_yscale("log")
ax1.set_ylim(10, 1e5)
ax1.set_title("CMS Open Data\n~270M processed events, 144GB")
ax2.set_xlabel("$\sum pT$ [GeV]")
ax1.set_ylabel("Events / bin")
ax2.set_ylabel("data / MC")
plt.savefig("../paper/plots/sumpt.pdf", bbox_inches="tight")
plt.savefig("../paper/plots/sumpt.png", bbox_inches="tight")